## Import library 

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import evaluation as evl
import pandas as pd
import numpy as np
import numexpr as ne

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


## Influence function 

In [2]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

RESULT_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/result"

In [3]:
delta = 2.5
index_ = 1
DATA_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/datasets/IHDP/csv/ihdp_npci_{}.csv".format(index_)
data = pd.read_csv(DATA_PATH, header = None, names = col)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(data[cov], data[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

influence = ipse.influence_function(data, treatment, cov, outcome, delta, model_y, model_t)

true_effect = data['mu1'] - data['mu0']
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

influence = ipse.influence_function(data, treatment, cov, outcome, delta, model_y, model_t)
means, stds = np.mean(influence, axis=0), sem(influence, axis=0)
print('Estimation effect: {:.3f}+-{:.3f}'.format(means, stds))


True effect: 4.016+-0.031
Estimation effect: 3.623+-0.278


## Sampling 

In [4]:
DATA_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/datasets/IHDP/csv/ihdp_npci_{}.csv".format(index_)
data = pd.read_csv(DATA_PATH, header = None, names = col)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(data[cov], data[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

num_of_delta = 5
delta_seq = np.linspace(0.5, 5, num_of_delta)

influence = ipse.sampling(data, treatment, cov, outcome, delta_seq, model_t, model_y)

100%|██████████| 5/5 [00:00<00:00,  8.11it/s]


In [5]:
true_effect = data['mu1'] - data['mu0']
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))
means, stds = np.mean(influence, axis=0), sem(influence, axis=0)
print('Estimation effect: {:.3f}+-{:.3f}'.format(means, stds))

True effect: 4.016+-0.031
Estimation effect: 3.630+-0.278


## Dowhy

In [6]:
index_ = 10

In [7]:

DATA_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/datasets/IHDP/csv/ihdp_npci_{}.csv".format(index_)
data = pd.read_csv(DATA_PATH, header = None, names = col)
# Create a causal model from the data and given graph.
model = CausalModel(data=data,
                    treatment=treatment,
                    outcome=outcome,
                    common_causes=cov, proceed_when_unidentifiable=True)

data[treatment] = [bool(x) for x in data[treatment]]
# Identify causal effect and return target estimands
identified_estimand = model.identify_effect()

# Estimate the target estimand using a statistical method.
estimate = model.estimate_effect(
    identified_estimand, method_name="backdoor.propensity_score_weighting")

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/skl

In [8]:
print("Causal Estimate is " + str(estimate.value))
true_effect = data['mu1'] - data['mu0']
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

Causal Estimate is 2.3695480801689293
True effect: 4.586+-0.325


## Do experiment with 10 ihdp dataset
**Do experiment and compare the result with different algorithms: stratification, matching, weighting, linear regression**

In [9]:
df_result = pd.DataFrame()
df_result['true_effect'] = 0
df_result['linear_regression'] = 0
df_result['weighting'] = 0
df_result['stratification'] = 0
df_result['incremental'] = 0


In [10]:
for index_ in tqdm(range(1,11)):    
    DATA_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/datasets/IHDP/csv/ihdp_npci_{}.csv".format(index_)
    data = pd.read_csv(DATA_PATH, header = None, names = col)

    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    num_of_delta = 5
    delta_seq = np.linspace(0.5, 5, num_of_delta)
    influence = ipse.sampling(data, treatment, cov, outcome, delta_seq, model_t, model_y)
    
    ### Weighting 
    model = CausalModel(data=data,
                    treatment=treatment,
                    outcome=outcome,
                    common_causes=cov, proceed_when_unidentifiable=True)
    data[treatment] = [bool(x) for x in data[treatment]]
    identified_estimand = model.identify_effect()
    estimate_we = model.estimate_effect(
        identified_estimand, method_name="backdoor.propensity_score_weighting")
    
    ### Stratification
    estimate_sf = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification", 
                                     method_params={'num_strata':50, 'clipping_threshold':5})
    

    
    ### Stratification
    estimate_lr = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", test_significance=True)


    means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    new_row = {'true_effect': round(means,2), 
               'linear_regression': round(estimate_lr.value,2), 
               'weighting':round(estimate_we.value,2), 
               'stratification':round(estimate_sf.value,2),
               'incremental':round(means_incre,2),
              }
    df_result = df_result.append(new_row, ignore_index=True)

df_result.to_csv(RESULT_PATH + "/result_10_ihdp.csv", index = False)

100%|██████████| 5/5 [00:00<00:00,  8.11it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.cond

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.740
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     78.91
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          6.10e-191
Time:                        18:38:35   Log-Likelihood:                -1138.1
No. Observations:                 747   AIC:                             2330.
Df Residuals:                     720   BIC:                             2455.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2206      0.448      4.957      0.0


100%|██████████| 5/5 [00:00<00:00,  8.06it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.con

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     80.18
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          8.88e-193
Time:                        18:38:40   Log-Likelihood:                -1101.5
No. Observations:                 747   AIC:                             2257.
Df Residuals:                     720   BIC:                             2382.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.4382      0.427      8.061      0.0


100%|██████████| 5/5 [00:00<00:00,  7.98it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.con

...

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                     77.94
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          1.58e-189
Time:                        18:39:12   Log-Likelihood:                -1273.2
No. Observations:                 747   AIC:                             2600.
Df Residuals:                     720   BIC:                             2725.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3494      0.537      2.514      0.0


100%|██████████| 5/5 [00:00<00:00,  8.02it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.con

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.710
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     67.72
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          7.98e-174
Time:                        18:39:18   Log-Likelihood:                -3006.9
No. Observations:                 747   AIC:                             6068.
Df Residuals:                     720   BIC:                             6193.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.9878      5.467      2.376      0.0


100%|██████████| 5/5 [00:00<00:00,  7.95it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.con

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     92.48
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          1.76e-209
Time:                        18:39:24   Log-Likelihood:                -2124.9
No. Observations:                 747   AIC:                             4304.
Df Residuals:                     720   BIC:                             4428.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.8970      1.679      3.513      0.0

## Do experiment with 100 ihdp dataset

In [11]:
def load_data(path, i_exp):
    col =  ["t", "yf", "ycf", "mu0", "mu1" ]
    cov = ["x" + str(i) for i in range(1,26)]
    col = col + cov
    features = cov + ["t"] 
    D = np.load(path)
    df = pd.DataFrame(columns=col)

    for i in range(1,26):
        df['x' + str(i)]  = D['x'][:,i-1,i_exp-1]

    df['t']  = D['t'][:,i_exp-1:i_exp]
    df['yf'] = D['yf'][:,i_exp-1:i_exp]
    df['ycf'] = D['ycf'][:,i_exp-1:i_exp]
    df['mu0'] = D['mu0'][:,i_exp-1:i_exp]
    df['mu1'] = D['mu1'][:,i_exp-1:i_exp]
    return df

In [12]:
PATH_TRAIN = "/home/dtd/Downloads/ihdp_npci_1-100.train.npz"

In [13]:
df_result = pd.DataFrame()
df_result['true_effect'] = 0
df_result['linear_regression'] = 0
df_result['weighting'] = 0
df_result['stratification'] = 0
df_result['incremental'] = 0



for index_ in tqdm(range(1,101)):    
    data = load_data(PATH_TRAIN, index_)

    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    num_of_delta = 5
    delta_seq = np.linspace(0.5, 5, num_of_delta)
    influence = ipse.sampling(data, treatment, cov, outcome, delta_seq, model_t, model_y)
    
    ### Weighting 
    model = CausalModel(data=data,
                    treatment=treatment,
                    outcome=outcome,
                    common_causes=cov, proceed_when_unidentifiable=True)
    data[treatment] = [bool(x) for x in data[treatment]]
    identified_estimand = model.identify_effect()
    
    estimate_we = model.estimate_effect(
        identified_estimand, method_name="backdoor.propensity_score_weighting")
    
    ### Stratification
    estimate_sf = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification", 
                                     method_params={'num_strata':50, 'clipping_threshold':5})

    ### Linear Regression
    estimate_lr = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", test_significance=True)


    means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    
    new_row = {'true_effect': round(means,2), 
               'linear_regression': round(estimate_lr.value,2), 
               'weighting':round(estimate_we.value,2), 
               'stratification':round(estimate_sf.value,2),
               'incremental':round(means_incre,2),
              }
    
    df_result = df_result.append(new_row, ignore_index=True)

df_result.to_csv(RESULT_PATH + "/result_100_ihdp.csv", index = False)

100%|██████████| 5/5 [00:00<00:00,  8.31it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.cond

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                     70.11
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          1.02e-168
Time:                        18:39:29   Log-Likelihood:                -1026.0
No. Observations:                 672   AIC:                             2106.
Df Residuals:                     645   BIC:                             2228.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3319      0.475      4.909      0.0


100%|██████████| 5/5 [00:00<00:00,  8.34it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.con

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     72.80
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          1.41e-172
Time:                        18:39:35   Log-Likelihood:                -989.19
No. Observations:                 672   AIC:                             2032.
Df Residuals:                     645   BIC:                             2154.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.5241      0.446      7.897      0.0


100%|██████████| 5/5 [00:00<00:00,  8.39it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.con

...

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     72.63
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          2.46e-172
Time:                        18:48:21   Log-Likelihood:                -2172.2
No. Observations:                 672   AIC:                             4398.
Df Residuals:                     645   BIC:                             4520.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.0249      2.618      2.301      0.0


100%|██████████| 5/5 [00:00<00:00,  8.30it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.con

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     92.50
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          4.92e-198
Time:                        18:48:26   Log-Likelihood:                -1442.3
No. Observations:                 672   AIC:                             2939.
Df Residuals:                     645   BIC:                             3060.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1459      0.902      4.594      0.0


100%|██████████| 5/5 [00:00<00:00,  8.36it/s]
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x25', 'x9', 'x1', 'x13', 'x23', 'x2', 'x10', 'x6', 'x22', 'x8', 'x14', 'x4', 'x3', 'x15', 'x16', 'x12', 'x24', 'x5', 'x11', 'x17', 'x20', 'x7', 'x21', 'x19', 'U', 'x18']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x25+x9+x1+x13+x23+x2+x10+x6+x22+x8+x14+x4+x3+x15+x16+x12+x24+x5+x11+x17+x20+x7+x21+x19+x18
/home/dtd/.con

                            OLS Regression Results                            
Dep. Variable:                     yf   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     87.32
Date:                Tue, 23 Jun 2020   Prob (F-statistic):          8.85e-192
Time:                        18:48:32   Log-Likelihood:                -1190.2
No. Observations:                 672   AIC:                             2434.
Df Residuals:                     645   BIC:                             2556.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6905      0.609      6.060      0.0

In [14]:
---------

SyntaxError: invalid syntax (<ipython-input-14-6676ce5d554b>, line 1)

## Do experiment with 1000 ihdp dataset

In [ ]:
df_result = pd.DataFrame()
df_result['true_effect'] = 0
df_result['linear_regression'] = 0
df_result['weighting'] = 0
df_result['stratification'] = 0
df_result['incremental'] = 0

PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-1000.train.npz"

for index_ in tqdm(range(1,1001)):    
    data = load_data(PATH_TRAIN, index_)

    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    num_of_delta = 5
    delta_seq = np.linspace(0.5, 5, num_of_delta)
    influence = ipse.sampling(data, treatment, cov, outcome, delta_seq, model_t, model_y)
    
    ### Weighting 
    model = CausalModel(data=data,
                    treatment=treatment,
                    outcome=outcome,
                    common_causes=cov, proceed_when_unidentifiable=True)
    data[treatment] = [bool(x) for x in data[treatment]]
    identified_estimand = model.identify_effect()
    estimate_we = model.estimate_effect(
        identified_estimand, method_name="backdoor.propensity_score_weighting")

    ### Stratification
    estimate_sf = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification", 
                                     method_params={'num_strata':50, 'clipping_threshold':5})
    

    
    ### Stratification
    estimate_lr = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", test_significance=True)

    
    means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    
    new_row = {'true_effect': round(means,2), 
               'linear_regression': round(estimate_lr.value,2), 
               'weighting':round(estimate_we.value,2), 
               'stratification':round(estimate_sf.value,2),
               'incremental':round(means_incre,2),
              }
    df_result = df_result.append(new_row, ignore_index=True)

df_result.to_csv(RESULT_PATH + "/result_1000_ihdp.csv", index = False)